In [9]:
!pip install torchvision==0.12.0
!pip install craft_text_detector
!pip install transformers

In [10]:
import cv2
import torch
from craft_text_detector import Craft
from google.colab import drive
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
craft = Craft(output_dir='output', crop_type='box', cuda=False)

In [15]:
image_path = '/content/drive/MyDrive/Screenshot 2024-09-20 233350.png'
prediction_result = craft.detect_text(image_path)

In [16]:
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-base-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-base-handwritten')


Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-base-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
for box in prediction_result['boxes']:
    box = [int(coord[0]) for coord in box]
    cropped_image = Image.open(image_path).crop(box)
    pixel_values = processor(images=cropped_image, return_tensors="pt").pixel_values
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]


ValueError: Coordinate 'lower' is less than 'upper'

In [ ]:
import spacy

# Load the pre-trained SciSpacy NER model
nlp = spacy.load("en_ner_bc5cdr_md")

# Sample text
text = generated_text

# Process the text with the model
doc = nlp(text)

# Extract and print named entities
for ent in doc.ents:
    print(ent.text, ent.label_)